## Mapper Agent

Action: Analyze the initial user input (question, idea, problem). Identify key entities, claims, and underlying assumptions.

Output structure: **Draft Knowledge Graph** -> A preliminary visual map showing the relationships between concepts in the *original prompt only*. (e.g. Idea A -> Assumes B)

Key Decision: The Mapper Agent reviews the prompt and determines if the requires deep research or not. If the prompt indicates that the Agent should simply try and understand the input and build a graph, it skips the deep research, otherwise, if the query requires additional knowledge then the Mapper Agent sends the prompt to a Research Agent.

In [ ]:
from openai import AsyncOpenAI
from agents import Agent, trace, Runner, function_tool, OpenAIChatCompletionsModel
from agents.model_settings import ModelSettings
from pydantic import BaseModel, Field, conlist
import asyncio
from typing import Dict
import requests
import os
from datetime import datetime
from IPython.display import Markdown, display

In [9]:
# Models

# get the API key
google_api_key = os.getenv("GOOGLE_API_KEY")

# connect to endpoints (necessary for Google but not OpenAI)
GEMINI_BASE_URL = "https://generativelanguage.googleapis.com/v1beta/openai/"

gemini_client = AsyncOpenAI(base_url=GEMINI_BASE_URL, api_key=google_api_key)

gemini_model = OpenAIChatCompletionsModel(model="gemini-2.0-flash", openai_client=gemini_client)

### Step 1: Create an Evaluator Agent that evaluates the prompt

Action: Review the prompt and determine if it requires deep research.

In [79]:
# Old Evaluator Agent instructions
old_EVALUATOR_AGENT_INSTRUCTIONS = """You are an prompt engineering analyst that is an expert at evaluating prompts.
You goal is evaluate a user's prompt and determine if the response requires a web search for more information. 
If the prompt requires a web search then you will pass it to the 'Planner Agent' who will develop a web search plan.
If the prompt does NOT require a web search then you will pass it to the 'Mapper Agent'.
"""
# New Evaluator Agent instructions
EVALUATOR_AGENT_INSTRUCTIONS = """You are an prompt engineering analyst that is an expert at evaluating prompts.
You goal is evaluate a user's prompt and determine if the response requires a web search for more information. 
If the prompt requires a web search then you will return True.
If the prompt does NOT require a web search then you will return False.
"""

evaluator_agent = Agent(
    name="Evaluator Agent",
    instructions=EVALUATOR_AGENT_INSTRUCTIONS,
    # handoffs=handoffs,
    model=gemini_model,
    output_type=bool,
)

In [80]:
# Let's set up a prompt for testing
async def evaluate_prompt(input: str) -> str:
    """Use the Evaluator Agent to determine if a prompt requires a web search or not"""
    result = await Runner.run(evaluator_agent, input)
    return result.final_output

In [13]:
# Set up some prompts for testing
web_search_required_prompt = f"What is the latest news for the Ukraine/ Russia war as of \
    today {datetime.now().strftime('%Y-%m-%d')}?"
    
no_web_search_required_prompt = "What are the names of the 7 continents?"

In [81]:
# Test if it correctly returns that a web search is needed
with trace("evaluator agent"):
    evaluation = await evaluate_prompt(web_search_required_prompt)
    print(evaluation)

True


In [82]:
# Test if it correctly returns that a web search is NOT needed
evaluation = await evaluate_prompt(no_web_search_required_prompt)
print(evaluation)

False


<h3>Step 2: Create a Planner Agent that plans web searches</h3>

<b>Action</b>: If the Evaluator Agent thinks that a web search is needed for more information in order to respond to the users prompt, then task the Planner Agent with creating a list of *N* number of web search queries.

<b>NOTE:</b> I found that when I was using string interpolation to identify the number of search terms to query, it was not actually working very well. The `planner_agent` was generating more than 3 queries (`WebSearchItems`). So I changed WebSearchPlan to explicitly hold *ONLY* 3 `WebSearchItems`.

<b>Original code:</b>
<blockquote style="padding-top: 10px; padding-bottom: 10px;">
class WebSearchPlan(BaseModel):<br>
&emsp;&emsp;searches: list[WebSearchItem] = Field(description="A list of web searches to perform to best answer the query.")
</blockquote>

<b>Modified code:</b>
<blockquote style="padding-top: 10px; padding-bottom: 10px;">
class WebSearchPlan(BaseModel):<br>
&emsp;&emsp;searches_1: WebSearchItem = Field(description="The first web searche to perform to best answer the query.")<br>
&emsp;&emsp;searches_2: WebSearchItem = Field(description="The second web searches to perform to best answer the query.")<br>
&emsp;&emsp;searches_3: WebSearchItem = Field(description="The third web searches to perform to best answer the query.")
</blockquote>


In [55]:
# Number of search terms to query for
NUM_SEARCHES = 3

# Planner Agent instructions
PLANNER_AGENT_INSTRUCTIONS = """You are a helpful research assistant. Given a query, come up with a set of web searches
to perform to best answer the query. Output {NUM_SEARCHES} terms to query for."""

# Structured outputs for web searches

class WebSearchItem(BaseModel):
    reason: str = Field(description="The reason why this search is important to the query.")
    
    query: str = Field(description="The search term used for the web search.")
    
class WebSearchPlan(BaseModel):
    searches_1: WebSearchItem = Field(description="The first web searche to perform to best answer the query.")
    searches_2: WebSearchItem = Field(description="The second web searches to perform to best answer the query.")
    searches_3: WebSearchItem = Field(description="The third web searches to perform to best answer the query.")
    
# Create a planner agent that outputs a WebSearchPlan
planner_agent = Agent(
    name="Planner Agent",
    instructions=PLANNER_AGENT_INSTRUCTIONS,
    model=gemini_model,
    output_type=WebSearchPlan,
)

In [52]:
# Let's test out this Planner Agent before going any further
async def plan_search(query: str) -> WebSearchPlan:
    """Use the Planner Agent to create a WebSearchPlan (list of WebSearchItems)"""
    result = await Runner.run(planner_agent, query)
    return result.final_output

In [24]:
# Create two different prompts to see what different WebSearchPlans look like

search_plan_prompt_1 = "What sort of beers become most popular during the holiday season?"

search_plan_prompt_2 = "Identify the best European cities to travel to around Christmas"

In [53]:
# Test 1
search_plan = await plan_search(search_plan_prompt_1)

In [39]:
def display_as_markdown_table(data_list):
    # 1. Start the Markdown table with headers
    markdown_output = "| Reason | Query |\n"
    markdown_output += "| :--- | :--- |\n"  # Separator line for alignment

    # 2. Add each row
    for item in data_list:
        # **CORRECTION**: Access elements by index [0] and [1] for a tuple
        try:
            reason = item[0] 
            query = item[1]
        except (IndexError, TypeError):
            # Fallback if the item isn't a tuple or doesn't have enough elements
            reason = "ERROR: Bad Format"
            query = "ERROR: Bad Format"

        # Escape pipe characters in the text to avoid breaking the table structure
        reason = str(reason).replace('|', '/')
        query = str(query).replace('|', '/')
        
        markdown_output += f"| {reason} | **{query}** |\n"

    # 3. Use the IPython display function to render the Markdown
    display(Markdown(markdown_output))

In [54]:
display_as_markdown_table(search_plan)

| Reason | Query |
| :--- | :--- |
| searches_1 | **reason='To understand the general categories and flavor profiles that align with holiday preferences.' query='holiday beers types'** |
| searches_2 | **reason='To identify specific beers that are commonly enjoyed or recommended during the holidays.' query='best holiday beers'** |
| searches_3 | **reason='To gather information about seasonal beer trends and consumer preferences during the holiday season.' query='popular winter beers'** |


In [60]:
# Test 2
search_plan = await plan_search(search_plan_prompt_2)

In [61]:
display_as_markdown_table(search_plan)

| Reason | Query |
| :--- | :--- |
| searches_1 | **reason='To identify popular European cities known for their Christmas markets and festive atmosphere.' query='best Christmas markets Europe'** |
| searches_2 | **reason='To find recommendations and rankings of European cities to visit during the Christmas season.' query='top European cities for Christmas travel'** |
| searches_3 | **reason='To get information about Christmas events, attractions, and overall travel experience in specific European cities during the holiday period.' query='Christmas in [European City]'** |


### Step 3: Create a Search Agent that executes specific web searches

##### Build Serper search function

In [57]:
@function_tool
def serper_search(query: str) -> str:
    """Peforms a web search using the Serper API and returns a structured string of results"""
    
    SERPER_API_KEY = os.getenv("SERPER_API_KEY")
    if not SERPER_API_KEY:
        return "Error: Serper API key not found."
    url = "https://google.serper.dev/search"
    payload = {"q": query}
    headers = {
        'X-API-KEY': SERPER_API_KEY,
        'Content-Type': 'application/json'
    }
    
    try:
        response = requests.post(url, headers=headers, json=payload)
        response.raise_for_status() # Raise an exception for bad status codes
        
        data = response.json()
        
        # --- Format the results for the Agent ---
        # The agent performs better with a concise, text-based summary.
        formatted_results = []
        
        # Prioritize 'organic' (general web) results
        if 'organic' in data:
            for item in data['organic'][:3]: # Take the top 3 results
                formatted_results.append(f"Title: {item.get('title')}\nSnippet: {item.get('snippet')}\nURL: {item.get('link')}")
                
        # Also include 'answerBox' or 'knowledgeGraph' for direct answers
        elif 'answerBox' in data and data['answerBox'].get('snippet'):
            formatted_results.insert(0, f"Direct Answer: {data['answerBox']['snippet']}")
        elif 'knowledgeGraph' in data and data['knowledgeGraph'].get('snippet'):
            formatted_results.insert(0, f"Direct Answer: {data['knowledgeGraph']['snippet']}")
            
        return "\n---\n".join(formatted_results) if formatted_results else "No relevant search results found."

    except requests.exceptions.RequestException as e:
        return f"Search Error: Failed to connect to Serper API. Details: {e}"
    

In [59]:

# Search Agent instructions
SEARCH_AGENT_INSTRUCTIONS = """You are a research assistant. Given a term you search the web for that term and
produce a concise summary of the results. The summary should be no more than 200 words.
Capture the main points. Write succinctly, no need to have complete sentences or good gramary. 
This will be consumed by another Agent synthesizing a report, so it's vital that you capture the
essence and ignore all the fluff. Do not include any additional commentary other than the summary itself.
"""

search_agent = Agent(
    name="Search Agent",
    instructions=SEARCH_AGENT_INSTRUCTIONS,
    tools=[serper_search],
    model=gemini_model,
    model_settings=ModelSettings(tool_choice="required"),
)


In [62]:
# Let's make a function for the Search Agent
async def search(item: WebSearchItem):
    """Use the search agent to run a web search for each item in the search plan"""
    input = f"Search item: {item.query}\nReason for searching: {item.reason}"
    result = await Runner.run(search_agent, input)
    return result.final_output

Now to test the search function, I am going to use the `WebSearchItem`s generated most recently that talk about holiday beer.

Reason	    -> Query<br>
searches_1	-> reason='To understand the general categories and flavor profiles that align with holiday preferences.' query='holiday beers types'<br>
searches_2	-> reason='To identify specific beers that are commonly enjoyed or recommended during the holidays.' query='best holiday beers'<br>
searches_3	-> reason='To gather information about seasonal beer trends and consumer preferences during the holiday season.' query='popular winter beers'


In [63]:
# Test searches_1
searches_1 = WebSearchItem(reason='To understand the general categories and flavor profiles that align with holiday preferences.', query='holiday beers types')

In [65]:
search_result = await search(searches_1)
print(search_result)

Holiday beers commonly include winter warmers (malt-forward, winter IPAs, spiced/flavored ales), rich stouts, and Christmas beers with seasonal flavors. Specific examples mentioned are Sierra Nevada Celebration IPA, Troegs Mad Elf Ale, St. Bernardus Christmas, and Two Roads Holiday Ale. These beers often aim for bold and rich profiles suited for cold weather.


In [66]:
display(Markdown(search_result))

Holiday beers commonly include winter warmers (malt-forward, winter IPAs, spiced/flavored ales), rich stouts, and Christmas beers with seasonal flavors. Specific examples mentioned are Sierra Nevada Celebration IPA, Troegs Mad Elf Ale, St. Bernardus Christmas, and Two Roads Holiday Ale. These beers often aim for bold and rich profiles suited for cold weather.

In [68]:
searches_2 = WebSearchItem(reason='To identify specific beers that are commonly enjoyed or recommended during the holidays.', query='best holiday beers')
search_result = await search(searches_2)
display(Markdown(search_result))

Holiday beer recommendations include: Winter IPAs such as Sierra Nevada Celebration Fresh Hop IPA, Hetty Alice Wintertime IPA, and Buoy Beer Strong Gale IPA. Other favorites include Great Lakes Christmas Ale, Sam Adams Old Fezziwig, St. Bernardus Christmas Ale, Schells Snowstorm, and Summit. Deschutes Jubelale is also consistently ranked as a top Christmas beer.

### Step 4: Connect Agents to complete search functionality

In [71]:
search_dict = search_plan.model_dump()
search_dict

{'searches_1': {'reason': 'To identify popular European cities known for their Christmas markets and festive atmosphere.',
  'query': 'best Christmas markets Europe'},
 'searches_2': {'reason': 'To find recommendations and rankings of European cities to visit during the Christmas season.',
  'query': 'top European cities for Christmas travel'},
 'searches_3': {'reason': 'To get information about Christmas events, attractions, and overall travel experience in specific European cities during the holiday period.',
  'query': 'Christmas in [European City]'}}

In [73]:
tasks = [asyncio.create_task(search(item)) for item in search_dict.values()]

In [75]:
tasks

[<Task finished name='Task-2962' coro=<search() done, defined at /var/folders/db/0cz282fj1cdbpwj6m82qcknc0000gn/T/ipykernel_69049/2048806457.py:2> exception=AttributeError("'dict' object has no attribute 'query'")>,
 <Task finished name='Task-2963' coro=<search() done, defined at /var/folders/db/0cz282fj1cdbpwj6m82qcknc0000gn/T/ipykernel_69049/2048806457.py:2> exception=AttributeError("'dict' object has no attribute 'query'")>,
 <Task finished name='Task-2964' coro=<search() done, defined at /var/folders/db/0cz282fj1cdbpwj6m82qcknc0000gn/T/ipykernel_69049/2048806457.py:2> exception=AttributeError("'dict' object has no attribute 'query'")>]

In [76]:
# Connect the Planner Agent to the Search Agent
async def perform_searches(search_plan: WebSearchPlan):
    """Call search agent for each item in the search plan """
    # Uncomment the line below if dynamic number of searchs in WebSearchPlan
    # tasks = [asyncio.create_task(search(item)) for item in search_plan.searches]
    
    # Hard coded the searches into a list since it's not dynamic
    search_items = [search_plan.searches_1, search_plan.searches_2, search_plan.searches_3]
    
    tasks = [asyncio.create_task(search(item)) for item in search_items]
    results = await asyncio.gather(*tasks)
    return results        

In [77]:
# Test the new function with the last WebSearchPlan: "search_plan"
searches_results = await perform_searches(search_plan)
print(searches_results)

['Top Christmas market destinations in Europe include cities in Germany, such as Aachen, Cologne, Düsseldorf, Dresden, Frankfurt, and Heidelberg. Krakow in Poland, Prague in the Czech Republic and Berlin in Germany are also noted as good options. Bavaria and Austria are also recommended.\n', 'Top European cities for Christmas travel include Strasbourg and Colmar in France, Nuremberg, Dresden, and Rothenburg ob der Tauber in Germany, Vienna in Austria, and Prague in the Czech Republic. Lisbon, Portugal is noted as a good city for solo travelers at Christmas, partially due to weather. Cologne, Germany and Paris, France are also good options.\n', "Christmas in European cities is characterized by festive Christmas markets in locations like Vienna's Schönbrunn palace and Strasbourg's Old Town. Key attractions include decorated housefronts, large Christmas trees, and a generally magical atmosphere. Cities such as Strasbourg, Nuremberg, Paris, and Cologne are highlighted as top destinations d

### Step 5: Mapper Agent.. Maybe?

Ohhh, I think I got it, the Mapper Agent can use the Evaluator Agent as a tool. It can be in addition to a Knowledge Graph tool.. potentially because we gotta see how that would work out.